<a href="https://colab.research.google.com/github/titaofdata/Week-10-Models-Used-in-Industries/blob/main/IntroCustomerSegmentation_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Segmentation

When it comes to finding out who your best customers are, the old RFM matrix principle is the best. 

RFM stands for *Recency*, *Frequency* and *Monetary.*   

It is a customer segmentation technique that uses past purchase behavior to divide customers into groups.  


### RFM Score Calculations  

**RECENCY (R)**: Days since last purchase  
**FREQUENCY (F)**: Total number of purchases  
**MONETARY VALUE (M)**: Total money this customer spent  

It is based on the marketing axiom that **80% of your business comes from 20% of your customers**.

RFM helps to identify customers who are more likely to respond to promotions by segmenting them into various categories.


### Import Packages

In [ ]:
# !pip install xlrd
# import package
import pandas as pd
import datetime

# suppress error warnings
import warnings
warnings.filterwarnings('ignore')

from google.colab import files

### Read Data

In [ ]:
# load dataset
data = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
#data = pd.read_excel('Online Retail.xlsx')

# Define Sales Column
data['Sales'] = data['Quantity'] * data['UnitPrice']
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


### Create RFM Segments

These are the the only 4 things we need to build our RFM segments:
1. *customers* : feature that specifies your users,
2. *dates* : dates of transactions
3. *transactions* : transaction number
4. *prices* : price of amount sold

In [ ]:
data.shape

(541909, 9)

In [ ]:
customers = 'CustomerID'
dates = 'InvoiceDate'
transactions = 'InvoiceNo'
prices = 'Sales'

In [ ]:
data['Description'] = data['Description'].str.strip()
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

data['InvoiceNo'] = data['InvoiceNo'].astype('str')
data = data[~data['InvoiceNo'].str.contains('C')]

In [ ]:
data.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Bahrain', 'Israel', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [ ]:
country = "United Kingdom"

# Filter for a single country
data_country = data[data.Country == country]

In [ ]:
data = data_country

NOW = datetime.datetime.now()

# RFM Feature Engineering
rfmTable = data.groupby(customers).agg({dates: lambda x: (NOW - x.max()).days, transactions: lambda x: len(x), prices: lambda x: x.sum()})
rfmTable[dates] = rfmTable[dates].astype(int)
rfmTable.rename(columns={dates: 'recency', 
                         transactions: 'frequency', 
                         prices: 'monetary_value'}, inplace=True)

In [ ]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,3251,1,77183.60
12747.0,2928,103,4196.01
12748.0,2926,4596,33719.73
12749.0,2929,199,4090.88
12820.0,2929,59,942.34


In [ ]:
rfmTable.shape

(3921, 3)

### Segment Users Based on Quantiles 

The easiest way to split metrics into segments is by using quartiles.  

1. This gives us a starting point for the detailed analysis.
2. 4 segments are easy to understand and explain.


In [ ]:
quantiles = rfmTable.describe()
quantiles

,recency,frequency,monetary_value
count,3921.000000,3921.000000,3921.000000
mean,3017.722265,90.371079,1863.910113
std,99.528532,217.796155,7481.922217
min,2926.000000,1.000000,0.000000
25%,2943.000000,17.000000,300.040000
50%,2976.000000,41.000000,651.820000
75%,3068.000000,99.000000,1575.890000
max,3299.000000,7847.000000,259657.300000


In [ ]:
## RFM scorer
segmented_rfm = rfmTable.copy()

def RScore(x, p, d):
    if x <= d[p]['25%']:
        return 1
    elif x <= d[p]['50%']:
        return 2
    elif x <= d[p]['75%']: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p]['25%']:
        return 4
    elif x <= d[p]['50%']:
        return 3
    elif x <= d[p]['75%']: 
        return 2
    else:
        return 1

### Score Users

In [ ]:
### Score each user
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency', quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

### Merge Scores
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm['RFMScore'] = segmented_rfm['RFMScore'].map(int)

segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,3251,1,77183.60,4,4,1,441
12747.0,2928,103,4196.01,1,1,1,111
12748.0,2926,4596,33719.73,1,1,1,111
12749.0,2929,199,4090.88,1,1,1,111
12820.0,2929,59,942.34,1,2,2,122


In [ ]:
data.to_csv('some_data.csv')

In [ ]:

segmented_rfm.to_csv('some_file_name.csv')

files.download('some_file_name.csv')

## Identify Customer Segments

### Who are the top 10 of our best customers?  
RFM Score of *111*.

In [ ]:
segmented_rfm['RFMScore'].sort_values().unique()

array([111, 112, 113, 114, 121, 122, 123, 124, 131, 132, 133, 134, 141,
       142, 143, 144, 211, 212, 213, 221, 222, 223, 224, 231, 232, 233,
       234, 241, 242, 243, 244, 311, 312, 313, 321, 322, 323, 324, 331,
       332, 333, 334, 341, 342, 343, 344, 411, 412, 413, 421, 422, 423,
       424, 431, 432, 433, 434, 441, 442, 443, 444])

In [ ]:
segmented_rfm[segmented_rfm['RFMScore']==111]

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12747.0,2928,103,4196.01,1,1,1,111
12748.0,2926,4596,33719.73,1,1,1,111
12749.0,2929,199,4090.88,1,1,1,111
12839.0,2928,314,5591.42,1,1,1,111
12841.0,2930,420,4022.35,1,1,1,111
...,...,...,...,...,...,...,...
18229.0,2937,164,7276.90,1,1,1,111
18241.0,2935,104,2073.09,1,1,1,111
18245.0,2933,175,2567.06,1,1,1,111


### Who are our Loyal Customers?  
Frequency score of *1*.

In [ ]:
segmented_rfm[segmented_rfm['f_quartile']==1].sort_values('frequency', ascending=False).tail(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
13637.0,2978,101,810.50,3,1,2,312
14584.0,3095,100,1042.26,4,1,2,412
15620.0,2982,100,1553.18,3,1,2,312
17614.0,2983,100,390.07,3,1,3,313
14217.0,2927,100,2004.98,1,1,1,111
17451.0,2927,100,1721.40,1,1,1,111
16031.0,3018,100,548.94,3,1,3,313
15113.0,2935,100,3609.33,1,1,1,111
15106.0,2943,100,1422.52,1,1,2,112


### Who are our Big Spenders?  
Monetary value score of *1*.

In [ ]:
segmented_rfm[segmented_rfm['m_quartile']==1].sort_values('monetary_value', ascending=False).tail(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
16496.0,2934,95,1599.52,1,2,1,121
13850.0,3055,93,1595.48,3,2,1,321
17837.0,2968,169,1593.38,2,1,1,211
16332.0,2954,153,1593.20,2,1,1,211
16676.0,2959,89,1592.12,2,2,1,221
14970.0,2994,34,1592.06,3,3,1,331
15611.0,2935,43,1591.45,1,2,1,121
18178.0,3054,97,1590.81,3,2,1,321
15532.0,2951,256,1580.93,2,1,1,211


### Who are almost lost?  
Haven't purchased for some time but spent a lot and transacted a lot.  
RFM Score of *311*.

In [ ]:
segmented_rfm[segmented_rfm['RFMScore']==311].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
16180.0,3026,162,10254.18,3,1,1,311
14952.0,2985,138,8099.49,3,1,1,311
16745.0,3012,357,7194.30,3,1,1,311
16652.0,2984,118,6773.97,3,1,1,311
17509.0,2983,366,6115.14,3,1,1,311
16984.0,3014,407,4481.35,3,1,1,311
15874.0,2989,120,4405.88,3,1,1,311
13555.0,2999,136,4149.28,3,1,1,311
15416.0,2990,192,3984.32,3,1,1,311


### Who are lost?  
Haven't purchased in the longest time but spent a lot and transacted a lot.  
RFM score of *411*.

In [ ]:
segmented_rfm[segmented_rfm['RFMScore']==411].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
13093.0,3201,159,7832.470,4,1,1,411
17850.0,3298,297,5391.210,4,1,1,411
15808.0,3232,208,3734.970,4,1,1,411
15379.0,3095,194,3703.290,4,1,1,411
13952.0,3143,137,3251.071,4,1,1,411
17504.0,3132,127,2997.030,4,1,1,411
12840.0,3069,113,2726.770,4,1,1,411
18260.0,3098,134,2643.200,4,1,1,411
16919.0,3082,327,2596.450,4,1,1,411


### Who are the bottom 10 worst customers?  
RFM score of *444*.

In [ ]:
segmented_rfm[segmented_rfm['RFMScore']==444].sort_values('monetary_value', ascending=False).tail(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
15940.0,3237,1,35.80,4,4,4,444
14576.0,3298,1,35.40,4,4,4,444
16765.0,3220,1,34.00,4,4,4,444
17408.0,3089,2,32.65,4,4,4,444
13120.0,3164,1,30.60,4,4,4,444
17102.0,3187,1,25.50,4,4,4,444
15823.0,3298,1,15.00,4,4,4,444
17763.0,3189,1,15.00,4,4,4,444
17956.0,3175,1,12.75,4,4,4,444


# Exercise: Do Customer Segmentation on a Different Dataset  
[link](https://query.data.world/s/oavhrdt2a4dhhg4agcbff6h3llnqsw) to dataset.